# Arize Demo Notebook (Simple monitoring)
This notebook gives an overview of how to send the Dummy Model CSV using the Python SDK.
 
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Arize-ai/client_python/blob/main/arize/examples/tutorials/My_First_KPI_Monitoring_Integration.ipynb)

In [ ]:
# Load Dummy Model CSV

import pandas as pd
import requests
import io

url = 'https://storage.googleapis.com/arize-assets/tutorials/fixture_data/mock_model_kpi.csv'
url_content=requests.get(url).content
df=pd.read_csv(io.StringIO(url_content.decode('utf-8')), low_memory=False)
df.head()

In [ ]:
## Install Arize & Initialize Client

!pip install arize -q

from arize.api import Client
import concurrent.futures as cf

API_KEY_STRING = 'YOUR_API_KEY_STRING'
ORGANIZATION_ID = 'YOUR_ORGANIZATION_ID'
arize = Client(organization_key=ORGANIZATION_ID, api_key=API_KEY_STRING)

## Important Points 
1. Predictions need a prediction ID. For now, I have just used the row ID in this file. Is there a batch/epoch id that you can send here?
2. Time - I have tweaked the time from the dummy value and instead sent in historical time values. 
3. We are not passing in any features right now. 
4. We are not passing in actuals right now. 

In [ ]:
# Processing to get Data into Right format for Arize 

# Prediction IDs (must be string typed) are one dataframe
df['prediction_id'] = df['prediction_id'].apply(str)

# Predictions (in this case, KPIs) are one dataframe 
kpi_df = df.filter(['kpi'], axis=1)

# Time is one dataframe (this is only necessary because we want to overwrite the timestamp to something reasonable). 
# Set the total_epoch_count to be the maximum number of epochs expected (can be in the ballpark)
# Assuming total epoch count is 1000, set the first timewrite to be 1000 hours before the current hour. 
total_epoch_count = 1000
import time
current_time = time.time()
def change_time(row_time):
    return int(current_time - ((total_epoch_count - row_time)*3600))
df['time'] = df['time'].apply(change_time)

df.head()

In [ ]:
# Log the 5000 Predictions and Actuals for the Dummy Model
# You can use either the Single Prediction Option or the Bulk Option. Both examples are below. 

# This is an example for logging single records. We discourage iterating in Python, if at time of logging you have a collection of records to log, we recommend using the bulk api (See example in the next cell). In this example we are only iterating through the first 20 items for brevity sake
futures = []
for i,k in enumerate(df['prediction_id'][:19]):
  future = arize.log_prediction(
        model_id='dummy_kpi_model',
        model_version='v1', 
        prediction_id=df['prediction_id'][i],
        prediction_label=df['kpi'][i],
        time_overwrite=int(df['time'][i]))
  futures.append(future)

# Listen to future to make sure record reached Arize
for response in cf.as_completed(futures):
  res = response.result()
  if res.status_code != 200:
    print(f'future failed because: {res.text}')

print('Done sending data to Arize!')


In [ ]:
# This example uses the bulk send.
pred_responses = arize.log_bulk_predictions(
      model_id='dummy_model',
      model_version='v1-bulk', 
      prediction_ids=df['prediction_id'],
      prediction_labels=kpi_df,
      time_overwrite=df['time'])
  
for response in cf.as_completed(pred_responses):
  res = response.result()
  print(f'predictions response completed... {res.status_code}')
  if res.status_code != 200:
    print(f'future failed because: {res.text}')

print('Done sending data to Arize!')
    